In [1]:
import json
import pandas as pd
import numpy as np
import datetime
from haversine import haversine

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))


In [2]:
df_sotf=pd.read_excel("/home/jian/BigLots/static_files/store_list_from_Dom/Store List Report 05.08.20 333PM.xlsx")
df_sotf=df_sotf[['Store','SOTF']]
df_sotf['SOTF']=df_sotf['SOTF'].apply(lambda x: str(x).lower().strip())
list_sotf=df_sotf[df_sotf['SOTF']=="yes"]['Store'].unique().tolist()
list_sotf=[str(x) for x in list_sotf]

In [3]:
df_store_locations=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200601-134325-120.txt",
                                 sep="|")
df_store_locations=df_store_locations[~df_store_locations['location_id'].isin([145,6990])]
df_store_locations=df_store_locations[['location_id','latitude_meas','longitude_meas']]
df_store_locations['location_id']=df_store_locations['location_id'].astype(str)
df_store_locations.loc[df_store_locations['location_id']=="5416",'latitude_meas']=38.9092794
df_store_locations.loc[df_store_locations['location_id']=="5416",'longitude_meas']=-78.1831506
df_store_locations['store_type']=np.where(df_store_locations['location_id'].isin(list_sotf),"SOTF","Legacy")
df_store_locations['coor']=df_store_locations[['latitude_meas','longitude_meas']].values.tolist()

In [4]:
dict_soft_location=df_store_locations[df_store_locations['store_type']=="SOTF"].set_index("location_id").to_dict()['coor']
dict_legacy_location=df_store_locations[df_store_locations['store_type']=="Legacy"].set_index("location_id").to_dict()['coor']

In [5]:
df_unique_zip=pd.read_excel("./BL_competitor_TuesdayMorning_Wave1_JL_2020-06-04.xlsx",dtype=str)

In [7]:
df_unique_zip['nearest_sotf']=np.nan
df_unique_zip['dist_to_nearest_sotf']=np.nan
df_unique_zip['dist_in_30_sotf']=np.nan
df_unique_zip['nearest_legacy']=np.nan
df_unique_zip['dist_to_nearest_legacy']=np.nan
df_unique_zip['dist_in_30_legacy']=np.nan

for ind,row in df_unique_zip.iterrows():
    zip_center=zip_centers[row['comp_target_zip']]
    dist_nearest_sotf=np.inf
    dist_nearest_legacy=np.inf
    
    nearest_sotf=np.nan
    nearest_legacy=np.nan
    
    for sotf, v in dict_soft_location.items():
        dist=haversine(zip_center,v,unit="mi")
        if dist<dist_nearest_sotf:
            dist_nearest_sotf=dist
            nearest_sotf=sotf
    for legacy, v in dict_legacy_location.items():
        dist=haversine(zip_center,v,unit="mi")
        if dist<dist_nearest_legacy:
            dist_nearest_legacy=dist
            nearest_legacy=legacy
    df_unique_zip.loc[ind,"nearest_sotf"]=nearest_sotf
    df_unique_zip.loc[ind,"dist_to_nearest_sotf"]=dist_nearest_sotf
    df_unique_zip.loc[ind,"dist_in_30_sotf"]=dist_nearest_sotf<=30
    df_unique_zip.loc[ind,"nearest_legacy"]=nearest_legacy
    df_unique_zip.loc[ind,"dist_to_nearest_legacy"]=dist_nearest_legacy
    df_unique_zip.loc[ind,"dist_in_30_legacy"]=dist_nearest_legacy<=30
    
    

In [8]:
df_unique_zip.shape

(7651, 7)

In [9]:
writer=pd.ExcelWriter("./updated_for_the_unique_zips_based_on_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zip.to_excel(writer,"df_unique_zip",index=False)
writer.save()